In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import imageio
import gc

from tqdm.auto import tqdm

import _pickle as pickle
import tensorflow as tf
def save(file,name, folder = ""):
    if folder != "":
        outfile = open('./'+folder+'/'+name+'.pickle', 'wb')
    else:
        outfile = open(name+'.pickle', 'wb')
    pickle.dump(file, outfile, protocol=4)
    outfile.close
    
def load(name, folder = ""):
    if folder != "":
        outfile = open('./'+folder+'/'+name+'.pickle', 'rb')
    else:
        outfile = open(name+'.pickle', 'rb')
    file = pickle.load(outfile)
    outfile.close
    return file

import cv2
from multiprocess import Pool
import tensorflow_hub as hub

In [ ]:
import tensorflow as tf

In [ ]:
tf.config.list_physical_devices()

In [ ]:
tf.__version__

In [ ]:
df =  pd.read_csv('train.csv')
df = df.sample(n = df.shape[0])
# df = pd.concat([df[df['label'] == i].sample(n = min(df[df['label'] == i].shape[0], 2500), replace = False) for i in range(5)])
# df = df.sample(n = df.shape[0])
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(df['image_id'].values, df['label'].values, test_size=0.2, random_state=42)

In [ ]:
df.head()

In [ ]:
df['label'].value_counts().to_dict()

In [ ]:
def proc(path2, im_size = 299):
    import imageio
    import cv2
    path, save_path = path2
    img = imageio.imread('./train_images/'+path)
    img = img.astype('float32')
    print(img.shape)
    img = cv2.resize(img, (800, 600))
    img = img.astype('uint8')
    imageio.imwrite(save_path + '/' + path, img)
    
train_inputs = list(zip(X_train, np.array(['./raw/train/'+str(elt) for elt in y_train])))
test_inputs = list(zip(X_test, np.array(['./raw/test/'+str(elt) for elt in y_test])))

p = Pool(10)
a = p.map(proc, test_inputs)
a = p.map(proc, train_inputs)
p.close()

In [ ]:
import resnet
build = resnet.ResnetBuilder()
model = build.build_resnet_18((299,299,3),5)

In [ ]:
inputs = tf.keras.Input(shape = (800,600,3))

x = hub.KerasLayer("https://tfhub.dev/tensorflow/efficientnet/b7/classification/1", trainable = True)(inputs)

# x = hub.KerasLayer("https://tfhub.dev/google/inaturalist/inception_v3/feature_vector/4",
#                    trainable=True)(inputs)

x = tf.keras.layers.Dense(5, activation = 'softmax')(x)

model = tf.keras.Model(inputs, x)

In [ ]:
model.load_weights('./checkpoint/weights.h5')
# model.load_weights('./checkpoint/efficient/weights')

In [ ]:
model.summary()

In [ ]:
batch_size = 1

train_data_generator = tf.keras.preprocessing.image.ImageDataGenerator(
                    rescale=1./255, rotation_range=20, zoom_range=0.15,
                         width_shift_range=0.2, height_shift_range=0.2, shear_range=0.15,
                         horizontal_flip=True, fill_mode="nearest")

test_data_generator = tf.keras.preprocessing.image.ImageDataGenerator(
                    rescale=1./255, shear_range=0, zoom_range=0,
                    horizontal_flip=False, rotation_range=0, width_shift_range=0,
                     height_shift_range=0)

train_datagen = tf.keras.preprocessing.image.DirectoryIterator(
                    './raw/train', train_data_generator, target_size=(800, 600),
                    batch_size=batch_size)

test_datagen = tf.keras.preprocessing.image.DirectoryIterator(
                    './raw/test',test_data_generator, target_size=(800, 600),
                    batch_size=batch_size)


In [ ]:
from tensorflow.keras.optimizers import Adam, SGD, RMSprop
model.compile(loss = tf.keras.losses.CategoricalCrossentropy(label_smoothing=0.1),
             optimizer = SGD(0.01),
#              optimizer = RMSprop(0.01),
#              optimizer = Adam(0.0001),
             metrics = ['accuracy'])

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.callbacks import ReduceLROnPlateau

early = EarlyStopping(monitor='val_loss', min_delta=0.0001, patience=8, verbose=1, 
                                                mode='min', restore_best_weights=True)

reduce = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=3, verbose=1, 
                           mode='min', min_delta=0.0001, cooldown=0, min_lr=0)

callbacks =[early, reduce]

epochs = 100
steps_per_epoch = int(8529/batch_size)
validation_steps = int(2133/batch_size)

# tg = train_gen()

history = model.fit(train_datagen, validation_data = test_datagen, epochs = epochs, 
                   steps_per_epoch = steps_per_epoch, validation_steps = validation_steps,
                   callbacks = callbacks)

In [ ]:
#  model.save_weights('./checkpoint/inception/weights')
model.save_weights('./checkpoint/weights.h5')

In [ ]:
c = 0
Y = []
pred = []
for x, y in test_datagen:
    if c >= 100:
        break
    else:
        p = model.predict(x)
        Y.append(y)
        pred.append(p)
        
    print(c)
    c+=1

In [ ]:
Y = np.concatenate(Y)
pred = np.concatenate(pred)

In [ ]:
Y1 = np.argmax(Y, axis = -1)
pred1 = np.argmax(pred, axis = -1)

In [ ]:
(pred1 == Y1).sum()/len(Y1)